In [4]:
from ansatz_simulation_class import AnsatzSimulation
import torch
from genetic_quantum import QuantumModel, QuanvLayer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
from patch_making import PatchExtraction, Quanv_2d
from torch import tensor
import matplotlib.pyplot as plt
from sampled_cv_dataset import SampledDataset4Training
import time
from tqdm import tqdm
import math

### Defining Classical Module class

In [5]:
class L2NormalizationLayer(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2NormalizationLayer, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

class HybridModel(nn.Module):
    def __init__(self, n_qubits, patch_size, chromosome, num_classes, input_size, ansatz_parameters, mode='2d'):
        super().__init__()
        self.quanv_layer = QuanvLayer(
            n_qubits=n_qubits,
            patch_size=patch_size,
            chromosome=chromosome,
            parameters = ansatz_parameters,
            mode=mode
        )
        feature_size = input_size
        self.fc1 = nn.Linear(feature_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.norm = nn.LayerNorm(feature_size)
        #self.conv1 = nn.Conv2d(in_channels = 1, output_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=patch_size, stride=patch_size)
        self.fc = nn.Linear(feature_size, num_classes)
        self.l2norm = L2NormalizationLayer(dim=1)
        
    def forward(self, x):
        print("Passing through quanvolution layer...")
        start_time = time.perf_counter()
        x = self.quanv_layer.forward(x)
        end_time = time.perf_counter()
        print(f'Quanvolution processing time: {end_time - start_time}')  
        x = x.flatten(start_dim=1)
        #print(x.shape)
        x = self.l2norm(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

### Instantiating genetic ansatz model

In [6]:
n_qubits = 4
max_patches = 80
patch_size = 2
batch_size = 100
num_classes = 2
input_size = max_patches * n_qubits
num_epochs = 4
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['ry_gate', 'rz_gate', 'ry_gate', 'rx_gate'],['trgt_0', 'ctrl_0', 'trgt_1', 'ctrl_1'], ['rz_gate', 'trgt_0', 'ctrl_0', 'ry_gate']]
#simple_fc = ClassicalComponent(num_classes, input_size)
params = torch.rand(6)*math.pi
hqcnn = HybridModel(n_qubits, patch_size, toy_chromosome, num_classes, input_size, params)
hqcnn

HybridModel(
  (quanv_layer): QuanvLayer()
  (fc1): Linear(in_features=320, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=2, bias=True)
  (l2norm): L2NormalizationLayer()
)

### Getting data and sampling it

In [11]:
def sample_classes(target_classes, dataset):
    return [dataset.__getitem__(index) for index in range(dataset.__len__()) if dataset.__getitem__(index)[1] in target_classes]

In [12]:
def sample_data(dataset, classes, sample_size):
    class_one = 0
    class_two = 0
    binary_dataset = []

    for image in dataset:
        if image[1] == classes[0] and class_one < sample_size:
            binary_dataset.append(image)
            class_one +=1 
        elif image[1] == [1] and class_two < sample_size:
            binary_dataset.append(image)
            class_two += 1
            
        if class_one == sample_size and class_two == sample_size:
            break
    
    return binary_dataset

In [14]:
def create_dataset(dataset_class, transform_routine, target_classes, sample_size):
    training_dataset = dataset_class(root='./data', train=True, download=True, transform=transform_routine)
    validation_dataset = dataset_class(root='./data', train=False, transform=transform_routine, download=True)
    
    reduced_training_dataset = sample_classes(target_classes, training_dataset)
    reduced_validation_dataset = sample_classes(target_classes, validation_dataset)
    
    sampled_training_dataset = sample_data(reduced_training_dataset, target_classes, sample_size)
    sampled_validation_dataset = sample_data(reduced_validation_dataset, target_classes, sample_size)
    
    return SampledDataset4Training(sampled_training_dataset), SampledDataset4Training(sampled_validation_dataset) 

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size, max_patches)])

target_classes = [0, 1]

sample_size = 200

training_data, validation_data = create_dataset(datasets.MNIST, transform, target_classes, sample_size)

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size, max_patches)])
mnist_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)


In [ ]:
mnist_validation_set = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

In [9]:
mnist_dataset.__getitem__(0)

(tensor([[0.9373, 0.9647, 0.9569, 0.8627],
         [0.8745, 1.0000, 0.9176, 0.8275],
         [0.8941, 0.9098, 0.8549, 0.9176],
         [0.8745, 0.9216, 0.8784, 0.8784],
         [0.8078, 1.0000, 0.8667, 0.8667],
         [0.9020, 0.9176, 0.7373, 0.9725],
         [0.9608, 0.8745, 0.8314, 0.8706],
         [0.9490, 0.9529, 0.8667, 0.7569],
         [0.9294, 0.8510, 0.8706, 0.8706],
         [0.8980, 0.9176, 0.8627, 0.8431],
         [0.8941, 0.8902, 0.8706, 0.8667],
         [0.8706, 0.8980, 0.9765, 0.7608],
         [0.8902, 0.9373, 0.8549, 0.8196],
         [0.9412, 0.8353, 0.8745, 0.8510],
         [0.8667, 0.8510, 0.8745, 0.8980],
         [0.8863, 0.8745, 0.8588, 0.8667],
         [0.8941, 0.8706, 0.8863, 0.8039],
         [0.8941, 0.8353, 0.8549, 0.8275],
         [0.8157, 0.8784, 0.8863, 0.8196],
         [0.8275, 0.8784, 0.6902, 0.9804],
         [0.7529, 0.8392, 0.8667, 0.9255],
         [0.8000, 0.8157, 0.7843, 0.9608],
         [0.7961, 0.8667, 0.8353, 0.8627],
         [0

In [33]:
len(mnist_dataset)

60000

In [10]:
selected_classes = [1, 4]
reduced_mnist = [mnist_dataset.__getitem__(index) for index in range(mnist_dataset.__len__()) if mnist_dataset.__getitem__(index)[1] in selected_classes]
len(reduced_mnist)

12000

In [35]:
mnist_dataset.__getitem__(4)

(tensor([[0.0000, 0.9059, 0.9922, 0.9882],
         [0.9882, 0.8902, 0.9059, 0.0000],
         [0.7725, 0.9922, 0.9843, 0.0000],
         [0.9922, 0.5412, 0.0941, 0.9882],
         [0.9216, 0.8510, 0.1647, 0.7529],
         [0.8235, 0.9882, 0.6588, 0.0000],
         [0.9451, 0.9882, 0.3020, 0.0000],
         [0.9882, 0.9922, 0.0000, 0.0000],
         [0.0000, 0.0627, 0.9882, 0.9882],
         [0.5804, 0.9922, 0.4431, 0.5804],
         [0.2157, 0.8235, 0.9922, 0.3412],
         [0.0000, 0.0000, 0.9098, 0.9922],
         [0.8863, 0.9882, 0.0000, 0.0000],
         [0.0549, 0.0000, 0.9882, 0.8824],
         [0.0549, 0.0000, 0.8431, 0.9882],
         [0.0000, 0.3412, 0.9882, 0.7412],
         [0.7176, 0.0000, 0.3608, 0.9882],
         [0.9882, 0.0000, 0.0000, 0.7882],
         [0.7137, 0.9922, 0.0000, 0.0000],
         [0.6902, 0.0000, 0.1412, 0.9882],
         [0.9922, 0.6902, 0.0000, 0.0314],
         [0.0157, 0.9490, 0.7451, 0.0196],
         [0.0000, 0.0000, 0.9922, 0.5725],
         [0

In [ ]:


class_one = 0
class_two = 0
binary_dataset = []

for image in reduced_mnist:
    if image[1] == 0 and class_one < 200:
        binary_dataset.append(image)
        class_one +=1 
    elif image[1] == 1 and class_two < 200:
        binary_dataset.append(image)
        class_two += 1
        
    if class_one == 200 and class_two == 200:
        break

In [37]:
images, labels = zip(*binary_dataset)
mnist_images = torch.stack(images)

In [38]:
mnist_images.shape

torch.Size([400, 50, 4])

In [39]:
mean = torch.mean(mnist_images)
mean

tensor(0.4163)

In [40]:
diffs = mnist_images - mean
var = torch.mean(torch.pow(diffs, 2.0))
var

tensor(0.1968)

In [41]:
std = torch.pow(var, 0.5)
std

tensor(0.4436)

### Training my model

In [16]:
#quantum_processing = transforms.Compose([Quanv_2d(n_qubits, toy_chromosome)])
#training_zeros_ones = SampledDataset4Training(binary_dataset)
training_loader = DataLoader(training_data, batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size, shuffle=True)

In [17]:
optimizer = torch.optim.Adam(hqcnn.parameters(), lr=0.1, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()
#loss = loss_fn(output, mnist_labels)
device = torch.device("cpu")

In [44]:
mean = 0.
meansq = 0.
for (data, label) in training_loader:
    mean = data.mean()
    meansq = (data**2).mean()

std = torch.sqrt(meansq - mean**2)
print("mean: " + str(mean))
print("std: " + str(std))

mean: tensor(0.4046)
std: tensor(0.4424)


In [45]:
post_processing = [data for (data, label) in training_loader]

In [46]:
post_process = torch.stack(post_processing).squeeze(0)
post_process.shape

torch.Size([4, 100, 50, 4])

In [47]:
mean_p = torch.mean(post_process)
mean_p

tensor(0.4163)

In [48]:
p_diffs = post_process - mean_p
p_var = torch.mean(torch.pow(p_diffs, 2.0))
p_var

tensor(0.1968)

In [49]:
p_std = torch.pow(p_var, 0.5)
p_std

tensor(0.4436)

In [50]:
phi_n = F.normalize(post_process.flatten(start_dim=1), dim=1)
cos_sim = phi_n @ phi_n.T
cos_sim.mean()

tensor(0.6441)

In [ ]:
def train_model(model, train_loader, num_epochs, optimizer, loss_fn, filepath):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        epoch_loss = 0.0
        total = 0
        
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in progress_bar:
        
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)  # Calculate loss
            
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            loss.backward()  # Backpropagate to calculate gradients
            optimizer.step() # Update weights
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/(i+1):.4f}, Acc: {correct / total:.4f}")
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss={epoch_loss:.4f}, Train Acc={epoch_acc:.4f}")
            # Print every 10 batches
        torch.save(model.state_dict(), filepath)    

In [53]:
torch.cuda.is_available()

False

In [ ]:
def test_model(model, test_loader, hyperparams, num_epochs, optimizer, loss_fn, output_file):
    model.eval()
    
    total_loss = 0.0
    all_labels = []
    all_predictions = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item(outputs, dim=1)[:, 1]
            
            probs = torch.softmax()
            all_probs.extend(probs)
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    avg_loss = total_loss / len(test_loader)
    
    with open(output_file, "a") as f:
        f.write(f"\n[Quantum Model Testing - {time.perf_counter()}]\n")
        f.write(f"Hyperparameters: {hyperparams}\n")
        f.write(f"Test Loss: {avg_loss:.4f}\n")
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"AUC: {auc:.4f}\n")
        f.write("-" * 50 + "\n")

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    return avg_loss, accuracy, precision, recall, f1, auc


In [ ]:
def validate_model(model, dataloader, criterion, device):
    model.eval()
    model.to(device)

    total_loss = 0.0
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
#            images = images.to(device)
#            labels = labels.squeeze().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy() if outputs.shape[1] > 1 else torch.softmax(outputs, dim=1)[:, 0].cpu().numpy()
            all_probs.extend(probs)
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    avg_loss = total_loss / len(dataloader)
    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    return avg_loss, accuracy, precision, recall, f1, auc



In [ ]:
#simple_fc = ClassicalComponent(num_classes, input_size)
filepath = "C:\Users\speak\Genetic-Algorithm-based-Optimization-for-Quantum-Circuit-Synthesis\src\models\hqcnn_weights.pth"

train_model(hqcnn, training_loader, num_epochs, optimizer, loss_fn)

Passing through quanvolution layer...
Quanvolution processing time: 2.356858999992255
Epoch [1/4], Step [1/4], Loss: 0.7029, Acc: 0.4800
Passing through quanvolution layer...
Quanvolution processing time: 2.103883000003407
Epoch [1/4], Step [2/4], Loss: 0.7010, Acc: 0.4850
Passing through quanvolution layer...
Quanvolution processing time: 2.0533047999924747
Epoch [1/4], Step [3/4], Loss: 0.6972, Acc: 0.5000
Passing through quanvolution layer...
Quanvolution processing time: 2.0809785000019474
Epoch [1/4], Step [4/4], Loss: 0.6976, Acc: 0.5000
Epoch 1: Train Loss=0.6976, Train Acc=0.5000
Passing through quanvolution layer...
Quanvolution processing time: 2.122514400005457
Epoch [2/4], Step [1/4], Loss: 0.7043, Acc: 0.4600
Passing through quanvolution layer...
Quanvolution processing time: 2.2003954000101658
Epoch [2/4], Step [2/4], Loss: 0.6977, Acc: 0.4900
Passing through quanvolution layer...
Quanvolution processing time: 2.687404600001173
Epoch [2/4], Step [3/4], Loss: 0.6979, Acc: 

In [54]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy().reshape(28, 28)
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

ValueError: cannot reshape array of size 200 into shape (28,28)

### Testing tensor dimensions and quantum convolution

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 4
dummy_input = torch.zeros(input_size, input_size, 2).numpy()
patched_images = [image.extract_patches_2d(img, (patch_size, patch_size)) for img in dummy_input]
#dummy_output = quanv.forward(dummy_input)

tensor(patched_images).shape

torch.Size([4, 3, 2, 2])

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 28

shape = (input_size, input_size)
dummy_input = np.zeros(shape, dtype=float)
patched_images = image.extract_patches_2d(dummy_input, (patch_size, patch_size)) 
#dummy_output = quanv.forward(dummy_input)
n_patches, patch_h, patch_w = tensor(patched_images).shape
patched_images = tensor(patched_images).contiguous().view(n_patches, patch_h*patch_w)
image._extract_patches()

In [ ]:
patched_images[0]

tensor([0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
patched_images.shape

torch.Size([729, 4])

In [ ]:
myAnsatz = AnsatzSimulation(n_qubits)
outputs = [myAnsatz.simulate_circuit(patch, 'rx', toy_chromosome) for patch in patched_images]

In [ ]:
tensor(outputs).shape

torch.Size([1458, 4])